<a href="https://colab.research.google.com/github/Rukkya/.py/blob/main/automata_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
class NFA:
    def __init__(self, num_states, transitions, start_state, accept_states):
        self.num_states = num_states
        self.transitions = transitions  # Format: {state: {symbol: [next_states]}}
        self.start_state = start_state
        self.accept_states = accept_states

    def epsilon_closure(self, states):
        closure = set(states)
        stack = list(states)
        while stack:
            state = stack.pop()
            if '' in self.transitions.get(state, {}):  # Epsilon transitions
                for next_state in self.transitions[state]['']:
                    if next_state not in closure:
                        closure.add(next_state)
                        stack.append(next_state)
        return closure


class DFA:
    def __init__(self):
        self.states = set()  # States are sets of NFA states (frozensets)
        self.transitions = {}  # Format: {frozenset(states): {symbol: frozenset(next_states)}}
        self.start_state = None
        self.accept_states = set()

    def add_transition(self, from_state, symbol, to_state):
        if from_state not in self.transitions:
            self.transitions[from_state] = {}
        self.transitions[from_state][symbol] = to_state


def nfa_to_dfa(nfa):
    dfa = DFA()
    initial_closure = frozenset(nfa.epsilon_closure({nfa.start_state}))
    dfa.start_state = initial_closure
    dfa.states.add(initial_closure)
    unprocessed_states = [initial_closure]

    while unprocessed_states:
        current_state = unprocessed_states.pop(0)
        # Iterate over all input symbols (excluding epsilon)
        for symbol in {s for trans in nfa.transitions.values() for s in trans if s != ''}:
            next_states = set()
            for nfa_state in current_state:
                if symbol in nfa.transitions.get(nfa_state, {}):
                    next_states.update(nfa.transitions[nfa_state][symbol])
            if next_states:
                closure = frozenset(nfa.epsilon_closure(next_states))
                if closure not in dfa.states:
                    dfa.states.add(closure)
                    unprocessed_states.append(closure)
                dfa.add_transition(current_state, symbol, closure)

                if any(state in nfa.accept_states for state in closure):
                    dfa.accept_states.add(closure)

    return dfa


# Example NFA Definition
nfa = NFA(
    num_states=3,
    transitions={
        0: {'a': [0, 1], 'b': [0]},
        1: {'b': [2]},
        2: {}
    },
    start_state=0,
    accept_states={2}
)

# Convert NFA to DFA
dfa = nfa_to_dfa(nfa)

# Print DFA States and Transitions
print("DFA States:")
for state in dfa.states:
    print(state)

print("\nDFA Transitions:")
for from_state, transitions in dfa.transitions.items():
    for symbol, to_state in transitions.items():
        print(f"{from_state} --{symbol}--> {to_state}")

print("\nDFA Start State:", dfa.start_state)
print("DFA Accept States:", dfa.accept_states)
